<a href="https://colab.research.google.com/github/erfanera/SlumImprovment/blob/main/demo/pyngrok_Dharavi_leaf_map_URL_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pyngrok

In [ ]:
!pip install streamlit pyngrok geopandas leafmap

In [45]:
from pyngrok import ngrok

# 終止所有现存的 ngrok 会话
ngrok.kill()

In [49]:
from pyngrok import ngrok
import streamlit as st
import leafmap.foliumap as leafmap
import geopandas as gpd
from shapely.geometry import Polygon
from PIL import Image
import numpy as np
import os

# 设置认证令牌
ngrok.set_auth_token("2fugZh9AN73a2NEVIPHPnVwXbID_5C42VDP8LV4sQsCsQ6eto")

# 创建 GeoDataFrame
def create_geodataframe_from_bbox(min_point, max_point):
    rectangle = Polygon([min_point, (max_point[0], min_point[1]), max_point, (min_point[0], max_point[1]), min_point])
    gdf = gpd.GeoDataFrame([1], geometry=[rectangle], columns=['dummy'])
    return gdf

# 划分图像为网格
def split_image_into_grid(image_path, grid_size):
    image = Image.open(image_path)
    img_array = np.array(image)
    height, width = img_array.shape[:2]
    cell_height = height // grid_size
    cell_width = width // grid_size
    grid_images = []
    for i in range(grid_size):
        for j in range(grid_size):
            top = i * cell_height
            bottom = (i + 1) * cell_height
            left = j * cell_width
            right = (j + 1) * cell_width
            grid_cell = image.crop((left, top, right, bottom))
            grid_images.append(grid_cell)
    return grid_images

def main():
    st.title("Custom Bounding Box and Grid Generator")

    # 地图和绘图工具
    m = leafmap.Map(draw_control=True, height=500)
    m.add_basemap("OpenStreetMap")
    m.to_streamlit(height=600)

    # 用户输入经纬度坐标
    min_lon = st.number_input("Enter minimum longitude:", value=72.8488)
    min_lat = st.number_input("Enter minimum latitude:", value=19.0384)
    max_lon = st.number_input("Enter maximum longitude:", value=72.8604)
    max_lat = st.number_input("Enter maximum latitude:", value=19.0484)

    # 创建边界框和显示地图
    gdf = create_geodataframe_from_bbox((min_lon, min_lat), (max_lon, max_lat))
    m.add_gdf(gdf, layer_name="Bounding Box")

    # 提取图像
    image_path = st.text_input("Enter path to image file:", "/root/satellite.tif")
    grid_size = st.number_input("Enter grid size (number of grids):", value=10)
    if st.button('Process Image'):
        grid_images = split_image_into_grid(image_path, grid_size)
        st.write("Here are the grid images:")
        for i, grid_image in enumerate(grid_images):
            st.image(grid_image, caption=f"Grid {i}")

    # 建立 ngrok 连接
    public_url = ngrok.connect(addr='8501')
    st.write('Streamlit is running at URL:', public_url)

if __name__ == "__main__":
    main()


In [50]:
from pyngrok import ngrok

# 启动 Streamlit 应用
os.system("streamlit run app.py &")

# 建立 ngrok 连接，确保使用正确的参数
public_url = ngrok.connect(addr='8501')
print('Streamlit is running at URL:', public_url)


Streamlit is running at URL: NgrokTunnel: "https://fea1-34-80-116-63.ngrok-free.app" -> "http://localhost:8501"
